In [7]:
import tensorflow as tf
from tf.keras.models import load_model 

TypeError: Couldn't build proto file into descriptor pool: duplicate symbol 'tensorflow.CoordinatedJob'

In [21]:
import requests
import time 
import numpy as np

In [ ]:
# Load the trained LSTM model
MODEL_PATH = "algae_lstm_model.h5"  
model = load_model(MODEL_PATH)

# Blynk credentials
BLYNK_AUTH_TOKEN = "IkotjlKBM5b8K_X_WXITua3U-WdwTeqb"
BLYNK_API_URL = "https://blynk.cloud/external/api"

def get_blynk_data():
    url = f"{BLYNK_API_URL}/get?token={BLYNK_AUTH_TOKEN}&V0&V1&V2&V3"
    response = requests.get(url).json()
    print("monitoring.....")
    print("Blynk Response:", response)  # Debugging
    
    # Ensure response is a dictionary with expected keys
    if isinstance(response, dict) and all(key in response for key in ["V0", "V1", "V2", "V3"]):
        try:
            sensor_values = [float(response[key]) for key in ["V0", "V1", "V2", "V3"]]
            
            # Ensure sensor values are not all zero (indicative of missing data)
            if all(val == 0 for val in sensor_values):
                print("Warning: Received all zero values from sensors. Skipping prediction.")
                return None
            
            return sensor_values
        except ValueError:
            print("Error: Received invalid data format from Blynk.")
            return None
    else:
        print("Error fetching data from Blynk:", response)
        return None
    
# Function to send alert to Blynk
def send_blynk_alert(message):
    if message == "Algae":
        value = 0
        time.sleep(2)

    else:
        value = 1
        url = f"{BLYNK_API_URL}/update?token={BLYNK_AUTH_TOKEN}&V6={value}"
        response = requests.get(url)
        print("Algae Alert Sent", response.text)
        log_url = f"{BLYNK_API_URL}/logEvent?token={BLYNK_AUTH_TOKEN}&code=algae_detection"
        log_response = requests.get(log_url)
        print("Notification Logged", log_response.text)
        
    time.sleep(2)
    reset_url = f"{BLYNK_API_URL}/update?token={BLYNK_AUTH_TOKEN}&V6=0"
    reset_response = requests.get(reset_url)
    print("", reset_response.text)


In [ ]:
# Main loop to fetch data and predict while True:
while True:
    real_time_data = get_blynk_data()
    
    if real_time_data is None:
        time.sleep(2)
        continue  # Skip this iteration if data is invalid
    
    input_data = np.array([real_time_data], dtype=np.float32).reshape(1, 4, 1)
    prediction = model.predict(input_data)[0][0]
    print(f"Prediction: {prediction}")

    # Send alert if algae bloom is predicted
    if prediction > 0.5:  # Assuming 0.5 as the threshold for bloom occurrence
        send_blynk_alert("Algae")
    
    time.sleep(5)  # Prevent API spamming

monitoring.....
Blynk Response: {'V0': 27.5, 'V1': 7.388, 'V2': 395.622, 'V3': 360.655}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
Prediction: 0.004426002502441406
monitoring.....
Blynk Response: {'V0': 27.5, 'V1': 7.388, 'V2': 395.622, 'V3': 360.655}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction: 0.004426002502441406


KeyboardInterrupt: 